In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import cv2
import os
from shutil import copyfile
import random
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, AveragePooling2D, Flatten, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

In [ ]:
train_mask_images = '../input/face-mask/image_data/training/mask/'
train_no_mask_images = '../input/face-mask/image_data/training/no_mask/'
valid_mask_images = '../input/face-mask/image_data/validation/mask/'
valid_no_mask_images = '../input/face-mask/image_data/validation/no_mask/'

In [ ]:
def neural_network(training_dir, validation_dir):
    model = Sequential()
    model.add(Conv2D(filters = 128, kernel_size = 2, activation = 'relu', input_shape = (200, 200, 3), data_format = 'channels_last'))
    model.add(MaxPooling2D(2))
    model.add(BatchNormalization())
#     model.add(Dropout(0.3))
    
    model.add(Conv2D(filters = 128, kernel_size = 2, activation = 'relu'))
    model.add(MaxPooling2D(2))
    model.add(BatchNormalization())  
#     model.add(Dropout(0.3))
    
    model.add(Conv2D(filters = 128, kernel_size = 2, activation = 'relu'))
    model.add(MaxPooling2D(2))
    model.add(BatchNormalization())  
#     model.add(Dropout(0.3))
    

    model.add(Flatten())
#     model.add(Dense(256, 'relu'))
#     model.add(BatchNormalization())
    model.add(Dense(128, 'relu'))
#     model.add(BatchNormalization())
    model.add(Dense(64, 'relu'))
#     model.add(BatchNormalization())
    model.add(Dense(1, 'sigmoid'))

    model.summary()
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor = 0.5, patience = 30, min_lr = 0.00001, verbose = 1)
    checkpoint = ModelCheckpoint('model.h5', monitor = 'loss', save_best_only = True)
    stop_early = EarlyStopping(monitor = 'loss', patience = 50)

    train_datagen = ImageDataGenerator(
      rotation_range=20,
      rescale = 1/255,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      fill_mode='nearest',
      horizontal_flip = True
      )
    
    validation_datagen = ImageDataGenerator(
      rotation_range=20,
      rescale = 1/255,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      fill_mode='nearest',
      horizontal_flip = True
      )

    train_generator = train_datagen.flow_from_directory(training_dir, batch_size = 32, class_mode = 'binary', target_size = (200,200))
    valid_generator = validation_datagen.flow_from_directory(validation_dir, batch_size = 32, class_mode = 'binary', target_size = (200,200))
    history = model.fit(train_generator, epochs = 500, verbose = 1, validation_data = valid_generator, callbacks = [checkpoint, reduce_lr, stop_early])

    return model, history

In [ ]:
model, history = neural_network('../input/face-mask/image_data/training/', '../input/face-mask/image_data/validation/')
model.save('version48_model')

In [ ]:
def make_plots(history):
    plt.figure()
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend(['Training', 'Validation'])
    
    plt.figure()
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend(['Training', 'Validation'])
    
make_plots(history)

In [ ]:
def test_prediction(filename, model):
    image = img_to_array(load_img(testing_folder + filename, target_size = (200, 200)) )
    image = np.reshape(image, (-1, 200, 200, 3))
    result = model.predict_classes(image)
    return result

In [ ]:
testing_folder = '../input/face-mask-testing-images/testing_images/'
model.load_weights('model.h5')

filenames = [] 

for file in sorted(os.listdir(testing_folder)):
    filenames.append(file)

labels = [0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,1,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,0,1,0,0,0,1,1,0]
df = pd.DataFrame()  
df['target'] = labels 
df.index = filenames
df['prediction'] = labels

for file in sorted(os.listdir(testing_folder)):
    df.loc[file, 'prediction'] = test_prediction(file, model)[0][0]

In [ ]:
print(classification_report(df.target, df.prediction))

In [ ]:
print(confusion_matrix(df.target, df.prediction))

In [ ]:
df